In [1]:
import loader as l
import plots as pl
import analysis as a
import patterns as p
import pandas as pd

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
tickers = 'CSCO'

In [3]:
stock_data = l.load_data(tickers)
stock_data = l.check_clean_data(stock_data, verbose=True)

From CSCO dataset were removed 6 rows with missing or incorrect data


In [4]:
stock_data[tickers]['splits']

Date
1996-02-20 00:00:00-05:00    2.0
1997-12-17 00:00:00-05:00    1.5
1998-09-16 00:00:00-04:00    1.5
1999-06-22 00:00:00-04:00    2.0
2000-03-23 00:00:00-05:00    2.0
Name: Stock Splits, dtype: float64

In [5]:
df = stock_data[tickers]['historical_data']
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1995-01-03 00:00:00-05:00,1.307,1.312,1.272,1.275,41986800,0.0,0.0
1995-01-04 00:00:00-05:00,1.284,1.298,1.265,1.284,54977400,0.0,0.0
1995-01-05 00:00:00-05:00,1.293,1.307,1.270,1.275,47637000,0.0,0.0
1995-01-06 00:00:00-05:00,1.270,1.289,1.256,1.275,36253800,0.0,0.0
1995-01-09 00:00:00-05:00,1.270,1.317,1.270,1.303,56772000,0.0,0.0
...,...,...,...,...,...,...,...
2024-06-28 00:00:00-04:00,47.150,47.450,46.940,47.110,23921100,0.0,0.0
2024-07-01 00:00:00-04:00,47.080,47.430,46.950,47.120,17776000,0.0,0.0
2024-07-02 00:00:00-04:00,47.000,47.380,46.560,46.880,19794200,0.0,0.0


In [6]:
pl.plot_chart(df, ticker=tickers)

In [7]:
support_df = a.calculate_ATR(df)

/Users/carlonestor/Documents/GitHub/statistical_analysis_candlestick_patterns/analysis.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
pattern_name = "Bullish Harami"  # replace with the desired pattern name

# check if the pattern exists in the patterns dictionary
if pattern_name in p.patterns:
    pattern_function = p.patterns[pattern_name]['function']
    # use the function to obtain the mask of the pattern dates
    mask = pattern_function(pd.concat([df, support_df], axis=1))
    # filter the mask to ensure consistency in the future analysis
    mask = p.filter_mask(mask, step=10)
    print(mask.sum())
else:
    print(f"The pattern '{pattern_name}' does not exist.")

93


In [9]:
pl.plot_close_with_patterns(df, tickers, mask)

In [10]:
pl.plot_patterns(df, mask, num_candles=p.patterns[pattern_name]['candles'], ticker=tickers, pattern_name=pattern_name)

In [11]:
returns = a.calculate_cumReturns_periods(df, mask)
log_returns = a.calculate_log_cumReturns_periods(df, mask)
# round the returns to 3 decimal places and multiply by 100 to make percentages more readable
rounded_returns = {k: [round(100*r,3) for r in v] for k, v in returns.items()}
print(returns.keys())
print(len(returns[1]))
print(returns[1])

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
93
[0.025661587810745633, 0.016335227272727366, -0.018806419257773363, 0.009773341651070848, -0.01173512154233027, -0.05272852866682019, -0.007704569606801253, 0.008890300047626615, 0.013703993735317206, 0.007312826465467279, 0.027675276752767593, 0.01783944499504457, 0.04547886570358469, -0.0306603773584905, 0.02244632400780746, -0.034318181818181775, -0.035914702581369126, 0.03710430721328489, 0.0941188545622498, 0.006284367635506683, -0.04416826003824106, 0.024315765663876932, 0.016765819361817324, -0.00960582974494863, 0.006253908692933061, 0.020771513353115677, -0.03874269005847949, 0.014119601328903797, -0.0160384923817162, -0.011063829787234108, 0.006240249609984405, 0.02227722772277239, 0.013077593722754889, -0.012121212121212133, -0.0068317677198975295, -0.0039062499999999167, 0.003791982665222117, -0.0040650406504064976, 0.0, 0.011249999999999982, 0.01546391752577317, -0.027027027027027035, -0.02178030303030307, -0.014781746031746121

In [12]:
pl.plot_return_distributions(rounded_returns)

In [13]:
avg_returns = a.calculate_average_return(rounded_returns)
median_returns = a.calculate_median_return(rounded_returns)
win_rate = a.calculate_win_rate(rounded_returns)

In [14]:
pl.plot_original_stats(avg_returns, median_returns, win_rate)

In [15]:
from scipy.stats import ks_2samp, anderson_ksamp

num_samples = len(rounded_returns[1]) if len(rounded_returns[1]) > 100 else 1000
num_samples = 150

bootstrap_returns = a.bootstrap_sample(rounded_returns, n_samples=num_samples)
kde_returns = a.kde_sample(rounded_returns, n_samples=num_samples)
bayesian_returns = a.bayesian_sample(rounded_returns, n_samples=num_samples)
mixed_returns = a.mixed_sample(rounded_returns, n_samples=num_samples)

Processing day 1


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


Processing day 2


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


...

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


Processing day 10


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [nu, sigma, mu]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 1 seconds.
Sampling: [likelihood]


In [16]:
results = []

for day in range(1,11):
    original_day = rounded_returns[day]
    bootstrap_day = bootstrap_returns[day]
    kde_day = kde_returns[day]
    bayesian_day = bayesian_returns[day]
    mixed_day = mixed_returns[day]

    # K-S Test
    ks_bootstrap = ks_2samp(original_day, bootstrap_day).pvalue
    ks_kde = ks_2samp(original_day, kde_day).pvalue
    ks_bayesian = ks_2samp(original_day, bayesian_day).pvalue
    ks_mixed = ks_2samp(original_day, mixed_day).pvalue
    # Anderson-Darling Test
    ad_bootstrap = anderson_ksamp([original_day, bootstrap_day]).significance_level
    ad_kde = anderson_ksamp([original_day, kde_day]).significance_level
    ad_bayesian = anderson_ksamp([original_day, bayesian_day]).significance_level
    ad_mixed = anderson_ksamp([original_day, mixed_day]).significance_level

    results.append({
        "day": day,
        "ks_bootstrap": ks_bootstrap,
        "ks_kde": ks_kde,
        "ks_bayesian": ks_bayesian,
        "ks_mixed": ks_mixed,
        "ad_bootstrap": ad_bootstrap,
        "ad_kde": ad_kde,
        "ad_bayesian": ad_bayesian,
        "ad_mixed": ad_mixed
    })

print(results)

[{'day': 1, 'ks_bootstrap': 0.9641834193672201, 'ks_kde': 0.8824096240104963, 'ks_bayesian': 0.9729943378241395, 'ks_mixed': 0.7923069988726109, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 2, 'ks_bootstrap': 0.9673088900777358, 'ks_kde': 0.9368927937334187, 'ks_bayesian': 0.6200970007298484, 'ks_mixed': 0.976145682399681, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 3, 'ks_bootstrap': 0.3910465860445811, 'ks_kde': 0.7643004680235812, 'ks_bayesian': 0.6169911460856035, 'ks_mixed': 0.8976144034568422, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 4, 'ks_bootstrap': 0.5842792223222646, 'ks_kde': 0.8473642274630057, 'ks_bayesian': 0.649993132797888, 'ks_mixed': 0.8976144034568422, 'ad_bootstrap': 0.25, 'ad_kde': 0.25, 'ad_bayesian': 0.25, 'ad_mixed': 0.25}, {'day': 5, 'ks_bootstrap': 0.9992222326161264, 'ks_kde': 0.8879410876224428, 'ks_bayesian': 0.9142263457544424, '

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_9826/297188851.py:16: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_9826/297188851.py:17: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_9826/297188851.py:18: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)

/var/folders/gr/66_s17nj0rq3hvgr_rys0w7m0000gn/T/ipykernel_9826/297188851.py:19: UserWarning:

p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)



In [17]:
pl.qq_plot(rounded_returns, kde_returns)

In [18]:
pl.qq_plot(rounded_returns, bayesian_returns)

In [19]:
pl.qq_plot(rounded_returns, bootstrap_returns)

In [20]:
pl.qq_plot(rounded_returns, mixed_returns)

In [21]:
pl.plot_return_distributions(kde_returns)

In [22]:
pl.plot_return_distributions(bayesian_returns)

In [23]:
pl.plot_return_distributions(mixed_returns)